In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# ========== CONFIG ==========
SEED = 42
np.random.seed(SEED)

# Base directory (same folder as this notebook)
DATA_DIR = Path(".")

# Input file paths
PRICE_FILE      = DATA_DIR / "henry_hub_01_01_10_to_02_12_25_biz.csv"
STORAGE_FILE    = DATA_DIR / "storage_daily_ffill.csv"
PRODUCTION_FILE = DATA_DIR / "us_dry_gas_production_daily_ffill.csv"
USD_FILE        = DATA_DIR / "usd_index_01_01_10_biz.csv"
WEATHER_FILE    = DATA_DIR / "weather_biz_dallas_daily.csv"

# Output file path
OUTPUT_FILE = DATA_DIR / "henryhub_master.csv"

print("Configuration loaded.")
print(f"Output will be saved to: {OUTPUT_FILE.resolve()}")

Configuration loaded.
Output will be saved to: /Users/adam/Documents/CS310 - 3rd Year Project/numeric_data/henryhub_master.csv


### Step 2 - Helper Function: `load_series`

We define a reusable helper function that:
- Reads a CSV file
- Parses the date column
- Sorts by date
- Drops duplicate dates (keeps first occurrence)

This ensures consistent preprocessing for all input datasets.

In [2]:
def load_series(path: Path, date_col: str = "date") -> pd.DataFrame:
    """
    Load a CSV time-series file with standard preprocessing.
    
    Parameters:
        path: Path to CSV file
        date_col: Name of the date column (default: 'date')
    
    Returns:
        DataFrame sorted by date with duplicates removed
    """
    df = pd.read_csv(path, parse_dates=[date_col])
    df = df.sort_values(date_col).reset_index(drop=True)
    
    # Drop duplicate dates, keeping the first occurrence
    n_before = len(df)
    df = df.drop_duplicates(subset=[date_col], keep="first").reset_index(drop=True)
    n_dropped = n_before - len(df)
    
    if n_dropped > 0:
        print(f"  [INFO] Dropped {n_dropped} duplicate date(s) from {path.name}")
    
    return df

print("Helper function defined.")

Helper function defined.


### Step 3 - Load & Rename Datasets

We load each CSV file and standardise column names:
- **Price**: `price`
- **Storage**: `storage_bcf`
- **Production**: `production_bcf`
- **USD Index**: `usd_index`
- **Weather**: keep existing names (`temp_c`, `temp_max_c`, `temp_min_c`, `hdd`, `cdd` if present)

Column name detection handles minor variations (e.g., `storage` vs `storage_bcf`).

In [3]:
def find_and_rename_column(df: pd.DataFrame, candidates: list, target_name: str) -> pd.DataFrame:
    """
    Find a column from a list of candidates and rename it to target_name.
    Raises an error if no candidate is found.
    """
    for col in candidates:
        if col in df.columns:
            if col != target_name:
                df = df.rename(columns={col: target_name})
                print(f"  [INFO] Renamed '{col}' -> '{target_name}'")
            return df
    raise ValueError(f"Could not find any of {candidates} in columns: {list(df.columns)}")

# ---------- PRICE ----------
print("Loading PRICE data...")
df_price = load_series(PRICE_FILE)
df_price = find_and_rename_column(df_price, ["price", "value", "Price", "VALUE"], "price")
df_price = df_price[["date", "price"]]
print(f"  Shape: {df_price.shape}, Date range: {df_price['date'].min()} to {df_price['date'].max()}")

# ---------- STORAGE ----------
print("\nLoading STORAGE data...")
df_storage = load_series(STORAGE_FILE)
df_storage = find_and_rename_column(df_storage, ["storage_bcf", "storage", "Storage", "STORAGE"], "storage_bcf")
df_storage = df_storage[["date", "storage_bcf"]]
print(f"  Shape: {df_storage.shape}, Date range: {df_storage['date'].min()} to {df_storage['date'].max()}")

# ---------- PRODUCTION ----------
print("\nLoading PRODUCTION data...")
df_production = load_series(PRODUCTION_FILE)
df_production = find_and_rename_column(df_production, ["production_bcf", "production", "Production", "PRODUCTION"], "production_bcf")
df_production = df_production[["date", "production_bcf"]]
print(f"  Shape: {df_production.shape}, Date range: {df_production['date'].min()} to {df_production['date'].max()}")

# ---------- USD INDEX ----------
print("\nLoading USD INDEX data...")
df_usd = load_series(USD_FILE)
df_usd = find_and_rename_column(df_usd, ["usd_index", "usd", "USD", "dxy", "DXY"], "usd_index")
df_usd = df_usd[["date", "usd_index"]]
print(f"  Shape: {df_usd.shape}, Date range: {df_usd['date'].min()} to {df_usd['date'].max()}")

# ---------- WEATHER ----------
print("\nLoading WEATHER data...")
df_weather = load_series(WEATHER_FILE)
# Keep all weather columns that exist
weather_cols = ["temp_c", "temp_max_c", "temp_min_c", "hdd", "cdd", "HDD", "CDD"]
available_weather_cols = [col for col in weather_cols if col in df_weather.columns]
# Standardise HDD/CDD column names to lowercase
rename_map = {"HDD": "hdd", "CDD": "cdd"}
df_weather = df_weather.rename(columns={k: v for k, v in rename_map.items() if k in df_weather.columns})
available_weather_cols = [rename_map.get(c, c) for c in available_weather_cols]
available_weather_cols = list(dict.fromkeys(available_weather_cols))  # remove duplicates, preserve order
df_weather = df_weather[["date"] + available_weather_cols]
print(f"  Shape: {df_weather.shape}, Weather columns: {available_weather_cols}")
print(f"  Date range: {df_weather['date'].min()} to {df_weather['date'].max()}")

print("\n✅ All datasets loaded successfully.")

Loading PRICE data...
  Shape: (4146, 2), Date range: 2010-01-04 00:00:00 to 2025-11-24 00:00:00

Loading STORAGE data...
  Shape: (5804, 2), Date range: 2010-01-01 00:00:00 to 2025-11-21 00:00:00

Loading PRODUCTION data...
  Shape: (5723, 2), Date range: 2010-01-01 00:00:00 to 2025-09-01 00:00:00

Loading USD INDEX data...
  Shape: (4156, 2), Date range: 2010-01-01 00:00:00 to 2025-12-05 00:00:00

Loading WEATHER data...
  Shape: (4156, 4), Weather columns: ['temp_c', 'temp_max_c', 'temp_min_c']
  Date range: 2010-01-01 00:00:00 to 2025-12-05 00:00:00

✅ All datasets loaded successfully.


### Step 4 - Merge Datasets

We perform a sequential **inner merge** on the `date` column, using the price dataset as the base.
This ensures we only keep dates that have data across all sources.

After merging, we:
- Sort by date
- Add an `id` column with constant value `"HH"` (Henry Hub identifier for TFT)

In [4]:
# Start with price as base
df_master = df_price.copy()
print(f"Base (price): {len(df_master)} rows")

# Merge storage
df_master = df_master.merge(df_storage, on="date", how="inner")
print(f"After merging storage: {len(df_master)} rows")

# Merge production
df_master = df_master.merge(df_production, on="date", how="inner")
print(f"After merging production: {len(df_master)} rows")

# Merge USD index
df_master = df_master.merge(df_usd, on="date", how="inner")
print(f"After merging USD index: {len(df_master)} rows")

# Merge weather
df_master = df_master.merge(df_weather, on="date", how="inner")
print(f"After merging weather: {len(df_master)} rows")

# Sort by date
df_master = df_master.sort_values("date").reset_index(drop=True)

# Add 'id' column for TFT (constant identifier)
df_master.insert(1, "id", "HH")

print(f"\n✅ Merge complete. Final shape: {df_master.shape}")
print(f"Columns: {list(df_master.columns)}")

Base (price): 4146 rows
After merging storage: 4145 rows
After merging production: 4086 rows
After merging USD index: 4086 rows
After merging weather: 4086 rows

✅ Merge complete. Final shape: (4086, 9)
Columns: ['date', 'id', 'price', 'storage_bcf', 'production_bcf', 'usd_index', 'temp_c', 'temp_max_c', 'temp_min_c']


### Step 5 - Data Quality Checks

We perform validation to ensure data integrity:
1. Print shape, head, and tail of the dataset
2. Assert dates are strictly increasing (no out-of-order rows)
3. Assert no duplicate dates exist
4. Report NaN counts per column and drop rows with NaN values

In [5]:
print("=" * 60)
print("DATA QUALITY CHECKS")
print("=" * 60)

# Shape
print(f"\n📊 Shape: {df_master.shape[0]} rows × {df_master.shape[1]} columns")

# Head
print("\n📋 First 5 rows:")
display(df_master.head())

# Tail
print("\n📋 Last 5 rows:")
display(df_master.tail())

# Check 1: Dates strictly increasing
print("\n🔍 Check 1: Dates strictly increasing...")
date_diffs = df_master["date"].diff().dropna()
is_strictly_increasing = (date_diffs > pd.Timedelta(0)).all()
assert is_strictly_increasing, "ERROR: Dates are NOT strictly increasing!"
print("   ✅ PASSED - All dates are strictly increasing.")

# Check 2: No duplicate dates
print("\n🔍 Check 2: No duplicate dates...")
n_duplicates = df_master["date"].duplicated().sum()
assert n_duplicates == 0, f"ERROR: Found {n_duplicates} duplicate date(s)!"
print("   ✅ PASSED - No duplicate dates found.")

# Check 3: NaN counts
print("\n🔍 Check 3: NaN counts per column...")
nan_counts = df_master.isna().sum()
total_nans = nan_counts.sum()
print(nan_counts.to_string())

if total_nans > 0:
    print(f"\n⚠️  Found {total_nans} total NaN value(s). Dropping affected rows...")
    rows_before = len(df_master)
    df_master = df_master.dropna().reset_index(drop=True)
    rows_after = len(df_master)
    print(f"   Dropped {rows_before - rows_after} row(s). New shape: {df_master.shape}")
else:
    print("   ✅ No NaN values found.")

print("\n" + "=" * 60)
print("✅ ALL DATA QUALITY CHECKS PASSED")
print("=" * 60)

DATA QUALITY CHECKS

📊 Shape: 4086 rows × 9 columns

📋 First 5 rows:


,date,id,price,storage_bcf,production_bcf,usd_index,temp_c,temp_max_c,temp_min_c
0,2010-01-04,HH,6.09,3117.0,1737233.0,92.3566,-4.2,-0.6,-7.8
1,2010-01-05,HH,6.19,3117.0,1737233.0,92.2236,-6.4,-1.7,-11.1
2,2010-01-06,HH,6.47,3117.0,1737233.0,92.0941,-2.2,2.8,-7.2
3,2010-01-07,HH,7.51,3117.0,1737233.0,92.3684,-4.4,0.6,-9.4
4,2010-01-08,HH,6.56,2850.0,1737233.0,92.1485,-8.3,-4.4,-12.2



📋 Last 5 rows:


,date,id,price,storage_bcf,production_bcf,usd_index,temp_c,temp_max_c,temp_min_c
4081,2025-08-26,HH,2.82,3217.0,3372895.0,120.9304,21.40,26.1,16.7
4082,2025-08-27,HH,2.88,3217.0,3372895.0,121.1236,19.20,21.7,16.7
4083,2025-08-28,HH,2.90,3217.0,3372895.0,120.6537,21.65,28.3,15.0
4084,2025-08-29,HH,2.88,3272.0,3372895.0,120.6028,16.95,18.3,15.6
4085,2025-09-01,HH,2.88,3272.0,3247480.0,120.6028,25.55,31.1,20.0



🔍 Check 1: Dates strictly increasing...
   ✅ PASSED - All dates are strictly increasing.

🔍 Check 2: No duplicate dates...
   ✅ PASSED - No duplicate dates found.

🔍 Check 3: NaN counts per column...
date              0
id                0
price             0
storage_bcf       0
production_bcf    0
usd_index         0
temp_c            0
temp_max_c        0
temp_min_c        0
   ✅ No NaN values found.

✅ ALL DATA QUALITY CHECKS PASSED


### Step 6 - Save Master CSV

We save the merged dataset to `henryhub_master.csv` with columns ordered as:
`date`, `id`, `price`, `storage_bcf`, `production_bcf`, `usd_index`, followed by weather columns.

The index is not saved to keep the CSV clean for model ingestion.

In [6]:
# Define column order
core_cols = ["date", "id", "price", "storage_bcf", "production_bcf", "usd_index"]
weather_cols_final = [c for c in df_master.columns if c not in core_cols]
final_col_order = core_cols + weather_cols_final

# Reorder columns
df_master = df_master[final_col_order]

# Save to CSV
df_master.to_csv(OUTPUT_FILE, index=False)

print(f"✅ Master CSV saved to: {OUTPUT_FILE.resolve()}")
print(f"   Final column order: {list(df_master.columns)}")

✅ Master CSV saved to: /Users/adam/Documents/CS310 - 3rd Year Project/numeric_data/henryhub_master.csv
   Final column order: ['date', 'id', 'price', 'storage_bcf', 'production_bcf', 'usd_index', 'temp_c', 'temp_max_c', 'temp_min_c']


### Step 7 - Final Confirmation

We verify the saved file and print a summary including:
- Output file path
- Date range covered
- Total row count
- File size

In [7]:
# Verify the saved file
df_verify = pd.read_csv(OUTPUT_FILE, parse_dates=["date"])

print("=" * 60)
print("FINAL CONFIRMATION")
print("=" * 60)
print(f"\n📁 Output file: {OUTPUT_FILE.resolve()}")
print(f"📅 Date range:  {df_verify['date'].min().strftime('%Y-%m-%d')} to {df_verify['date'].max().strftime('%Y-%m-%d')}")
print(f"📊 Total rows:  {len(df_verify):,}")
print(f"📐 Columns:     {len(df_verify.columns)} ({', '.join(df_verify.columns)})")

# File size
file_size_bytes = OUTPUT_FILE.stat().st_size
if file_size_bytes > 1024 * 1024:
    file_size_str = f"{file_size_bytes / (1024 * 1024):.2f} MB"
else:
    file_size_str = f"{file_size_bytes / 1024:.2f} KB"
print(f"💾 File size:   {file_size_str}")

print("\n" + "=" * 60)
print("🎉 MASTER DATASET BUILD COMPLETE!")
print("=" * 60)

FINAL CONFIRMATION

📁 Output file: /Users/adam/Documents/CS310 - 3rd Year Project/numeric_data/henryhub_master.csv
📅 Date range:  2010-01-04 to 2025-09-01
📊 Total rows:  4,086
📐 Columns:     9 (date, id, price, storage_bcf, production_bcf, usd_index, temp_c, temp_max_c, temp_min_c)
💾 File size:   245.21 KB

🎉 MASTER DATASET BUILD COMPLETE!
